In [2]:
from pyspark.sql import SparkSession, types

In [3]:
spark = SparkSession\
  .builder\
  .master("local[*]")\
  .appName('test')\
  .getOrCreate()

25/03/18 16:03:08 WARN Utils: Your hostname, Bastiens-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.30 instead (on interface en0)
25/03/18 16:03:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/18 16:03:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Green Taxi

In [4]:
green_schema = types.StructType([
  types.StructField('VendorID', types.IntegerType(), True),
  types.StructField('lpep_pickup_datetime', types.TimestampType(), True),
  types.StructField('lpep_dropoff_datetime', types.TimestampType(), True),
  types.StructField('store_and_fwd_flag', types.StringType(), True),
  types.StructField('RatecodeID', types.IntegerType(), True),
  types.StructField('PULocationID', types.IntegerType(), True),
  types.StructField('DOLocationID', types.IntegerType(), True),
  types.StructField('passenger_count', types.IntegerType(), True),
  types.StructField('trip_distance', types.DoubleType(), True),
  types.StructField('fare_amount', types.DoubleType(), True),
  types.StructField('extra', types.DoubleType(), True),
  types.StructField('mta_tax', types.DoubleType(), True),
  types.StructField('tip_amount', types.DoubleType(), True),
  types.StructField('tolls_amount', types.DoubleType(), True),
  types.StructField('ehail_fee', types.StringType(), True),
  types.StructField('improvement_surcharge', types.DoubleType(), True),
  types.StructField('total_amount', types.DoubleType(), True),
  types.StructField('payment_type', types.IntegerType(), True),
  types.StructField('trip_type', types.IntegerType(), True),
  types.StructField('congestion_surcharge', types.DoubleType(), True)
])

In [5]:
for year in range(2020, 2022):
  for month in range(1, 13):
    print(f"Processing data for {year}/{month:02d}")
    
    input_path = f'./data/raw/green/{year}/{month:02d}'
    output_path = f'./data/pq/green/{year}/{month:02d}'

    try:
      df_green = spark.read\
        .option("header", True)\
        .schema(green_schema)\
        .csv(input_path)

      df_green\
        .repartition(4)\
        .write.parquet(output_path, mode='overwrite')
    except:
      print(f"Could not process data for {year}/{month:02d}")

Processing data for 2020/01


Processing data for 2020/02


Processing data for 2020/03


Processing data for 2020/04
Processing data for 2020/05
Processing data for 2020/06
Processing data for 2020/07
Processing data for 2020/08
Processing data for 2020/09
Processing data for 2020/10
Processing data for 2020/11


Processing data for 2020/12


...

25/03/18 16:03:57 ERROR Executor: Exception in task 0.0 in stage 58.0 (TID 96)
java.io.EOFException: Unexpected end of input stream
	at org.apache.hadoop.io.compress.DecompressorStream.decompress(DecompressorStream.java:165)
	at org.apache.hadoop.io.compress.DecompressorStream.read(DecompressorStream.java:105)
	at java.base/java.io.InputStream.read(InputStream.java:205)
	at org.apache.hadoop.util.LineReader.fillBuffer(LineReader.java:191)
	at org.apache.hadoop.util.LineReader.readDefaultLine(LineReader.java:227)
	at org.apache.hadoop.util.LineReader.readLine(LineReader.java:185)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.skipUtfByteOrderMark(LineRecordReader.java:158)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.nextKeyValue(LineRecordReader.java:198)
	at org.apache.spark.sql.execution.datasources.RecordReaderIterator.hasNext(RecordReaderIterator.scala:39)
	at org.apache.spark.sql.execution.datasources.HadoopFileLinesReader.hasNext(HadoopFileLinesReader.sc

Could not process data for 2021/09
Processing data for 2021/10
Could not process data for 2021/10
Processing data for 2021/11


25/03/18 16:03:57 ERROR Executor: Exception in task 0.0 in stage 59.0 (TID 97)
java.io.EOFException: Unexpected end of input stream
	at org.apache.hadoop.io.compress.DecompressorStream.decompress(DecompressorStream.java:165)
	at org.apache.hadoop.io.compress.DecompressorStream.read(DecompressorStream.java:105)
	at java.base/java.io.InputStream.read(InputStream.java:205)
	at org.apache.hadoop.util.LineReader.fillBuffer(LineReader.java:191)
	at org.apache.hadoop.util.LineReader.readDefaultLine(LineReader.java:227)
	at org.apache.hadoop.util.LineReader.readLine(LineReader.java:185)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.skipUtfByteOrderMark(LineRecordReader.java:158)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.nextKeyValue(LineRecordReader.java:198)
	at org.apache.spark.sql.execution.datasources.RecordReaderIterator.hasNext(RecordReaderIterator.scala:39)
	at org.apache.spark.sql.execution.datasources.HadoopFileLinesReader.hasNext(HadoopFileLinesReader.sc

Could not process data for 2021/11
Processing data for 2021/12
Could not process data for 2021/12


25/03/18 16:03:57 ERROR Executor: Exception in task 0.0 in stage 61.0 (TID 99)
java.io.EOFException: Unexpected end of input stream
	at org.apache.hadoop.io.compress.DecompressorStream.decompress(DecompressorStream.java:165)
	at org.apache.hadoop.io.compress.DecompressorStream.read(DecompressorStream.java:105)
	at java.base/java.io.InputStream.read(InputStream.java:205)
	at org.apache.hadoop.util.LineReader.fillBuffer(LineReader.java:191)
	at org.apache.hadoop.util.LineReader.readDefaultLine(LineReader.java:227)
	at org.apache.hadoop.util.LineReader.readLine(LineReader.java:185)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.skipUtfByteOrderMark(LineRecordReader.java:158)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.nextKeyValue(LineRecordReader.java:198)
	at org.apache.spark.sql.execution.datasources.RecordReaderIterator.hasNext(RecordReaderIterator.scala:39)
	at org.apache.spark.sql.execution.datasources.HadoopFileLinesReader.hasNext(HadoopFileLinesReader.sc

## Yellow Taxi

In [6]:
yellow_schema = types.StructType([
  types.StructField('VendorID', types.IntegerType(), True),
  types.StructField('tpep_pickup_datetime', types.TimestampType(), True),
  types.StructField('tpep_dropoff_datetime', types.TimestampType(), True),
  types.StructField('passenger_count', types.IntegerType(), True),
  types.StructField('trip_distance', types.DoubleType(), True),
  types.StructField('RatecodeID', types.IntegerType(), True),
  types.StructField('store_and_fwd_flag', types.StringType(), True),
  types.StructField('PULocationID', types.IntegerType(), True),
  types.StructField('DOLocationID', types.IntegerType(), True),
  types.StructField('payment_type', types.IntegerType(), True),
  types.StructField('fare_amount', types.DoubleType(), True),
  types.StructField('extra', types.DoubleType(), True),
  types.StructField('mta_tax', types.DoubleType(), True),
  types.StructField('tip_amount', types.DoubleType(), True),
  types.StructField('tolls_amount', types.DoubleType(), True),
  types.StructField('improvement_surcharge', types.DoubleType(), True),
  types.StructField('total_amount', types.DoubleType(), True),
  types.StructField('congestion_surcharge', types.DoubleType(), True)
])

In [7]:
for year in range(2020, 2022):
  for month in range(1, 13):
    print(f"Processing data for {year}/{month:02d}")
    
    input_path = f'./data/raw/yellow/{year}/{month:02d}'
    output_path = f'./data/pq/yellow/{year}/{month:02d}'

    try:
      df_yellow = spark.read\
        .option("header", True)\
        .schema(yellow_schema)\
        .csv(input_path)

      df_yellow\
        .repartition(4)\
        .write.parquet(output_path, mode='overwrite')
    except:
      print(f"Could not process data for {year}/{month:02d}")

Processing data for 2020/01


Processing data for 2020/02


Processing data for 2020/03


Processing data for 2020/04


Processing data for 2020/05


...

25/03/18 16:08:37 ERROR Executor: Exception in task 0.0 in stage 119.0 (TID 195)
java.io.EOFException: Unexpected end of input stream
	at org.apache.hadoop.io.compress.DecompressorStream.decompress(DecompressorStream.java:165)
	at org.apache.hadoop.io.compress.DecompressorStream.read(DecompressorStream.java:105)
	at java.base/java.io.InputStream.read(InputStream.java:205)
	at org.apache.hadoop.util.LineReader.fillBuffer(LineReader.java:191)
	at org.apache.hadoop.util.LineReader.readDefaultLine(LineReader.java:227)
	at org.apache.hadoop.util.LineReader.readLine(LineReader.java:185)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.skipUtfByteOrderMark(LineRecordReader.java:158)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.nextKeyValue(LineRecordReader.java:198)
	at org.apache.spark.sql.execution.datasources.RecordReaderIterator.hasNext(RecordReaderIterator.scala:39)
	at org.apache.spark.sql.execution.datasources.HadoopFileLinesReader.hasNext(HadoopFileLinesReader.

Processing data for 2021/08
Could not process data for 2021/08
Processing data for 2021/09


25/03/18 16:08:37 ERROR Executor: Exception in task 0.0 in stage 120.0 (TID 196)
java.io.EOFException: Unexpected end of input stream
	at org.apache.hadoop.io.compress.DecompressorStream.decompress(DecompressorStream.java:165)
	at org.apache.hadoop.io.compress.DecompressorStream.read(DecompressorStream.java:105)
	at java.base/java.io.InputStream.read(InputStream.java:205)
	at org.apache.hadoop.util.LineReader.fillBuffer(LineReader.java:191)
	at org.apache.hadoop.util.LineReader.readDefaultLine(LineReader.java:227)
	at org.apache.hadoop.util.LineReader.readLine(LineReader.java:185)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.skipUtfByteOrderMark(LineRecordReader.java:158)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.nextKeyValue(LineRecordReader.java:198)
	at org.apache.spark.sql.execution.datasources.RecordReaderIterator.hasNext(RecordReaderIterator.scala:39)
	at org.apache.spark.sql.execution.datasources.HadoopFileLinesReader.hasNext(HadoopFileLinesReader.

Could not process data for 2021/09
Processing data for 2021/10
Could not process data for 2021/10
Processing data for 2021/11
Could not process data for 2021/11
Processing data for 2021/12
Could not process data for 2021/12


25/03/18 16:08:37 ERROR Executor: Exception in task 0.0 in stage 122.0 (TID 198)
java.io.EOFException: Unexpected end of input stream
	at org.apache.hadoop.io.compress.DecompressorStream.decompress(DecompressorStream.java:165)
	at org.apache.hadoop.io.compress.DecompressorStream.read(DecompressorStream.java:105)
	at java.base/java.io.InputStream.read(InputStream.java:205)
	at org.apache.hadoop.util.LineReader.fillBuffer(LineReader.java:191)
	at org.apache.hadoop.util.LineReader.readDefaultLine(LineReader.java:227)
	at org.apache.hadoop.util.LineReader.readLine(LineReader.java:185)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.skipUtfByteOrderMark(LineRecordReader.java:158)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.nextKeyValue(LineRecordReader.java:198)
	at org.apache.spark.sql.execution.datasources.RecordReaderIterator.hasNext(RecordReaderIterator.scala:39)
	at org.apache.spark.sql.execution.datasources.HadoopFileLinesReader.hasNext(HadoopFileLinesReader.

In [8]:
spark.stop()